## Imports

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import pdb

torch.manual_seed(69)

https://github.com/pytorch/examples/tree/master/vae

## Model

In [2]:
class VAE(nn.Module):
    def __init__(self, z):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(4096, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc21 = nn.Linear(1024, z)
        self.fc22 = nn.Linear(1024, z)
        self.fc3 = nn.Linear(z, 1024)
        self.fc32 = nn.Linear(1024, 1024)
        self.fc4 = nn.Linear(1024, 4096)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h1 = F.relu(self.fc2(h1))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        h3 = F.relu(self.fc32(h3))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 4096))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
    
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class UnFlatten(nn.Module):
    def __init__(self, h):
        super(UnFlatten, self).__init__() 
        self.h = h
        
    def forward(self, input):
        return input.view(input.size(0), self.h, 1, 1)
    
# https://github.com/sksq96/pytorch-vae/blob/master/vae-cnn.ipynb
class VAE_CNN(nn.Module):
    def __init__(self, image_channels=1, h_dim=1024, z_dim=32):
        super(VAE_CNN, self).__init__()
        self.h = h_dim
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2),
            nn.ReLU(),
            Flatten()
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            UnFlatten(self.h),
            nn.ConvTranspose2d(h_dim, 256, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=6, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, image_channels, kernel_size=6, stride=2),
            nn.Sigmoid(),
        )
        
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        return z, mu, logvar

    def decode(self, z):
        z = self.fc3(z)
        z = self.decoder(z)
        return z

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        z = self.decode(z)
        return z, mu, logvar


## Train/ Test

In [3]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x.view(-1, 4096), x.view(-1, 4096), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch, log_interval):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch, batch_size):
    model.eval()
    test_loss = 0
    loader = test_loader
    with torch.no_grad():
        for i, (data, _) in enumerate(loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0 and epoch % 5 == 0:
                n = min(data.size(0), 8)
                new_recon_batch = recon_batch.view(batch_size, 1, 64, 64)[:n]
                comparison = torch.cat([data[:n], new_recon_batch])
                save_image(comparison.cpu(),
                         'results_kanji/reconstruction_kanji_cnn_big_dubs_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


## Run

In [4]:
# Parameters
batch_size = 128
epochs = 15
log_interval = 400
h = 2048
z = 128

# Cuda
hasCuda = torch.cuda.is_available()
device = torch.device("cuda" if hasCuda else "cpu")

# Data
kwargs = {'num_workers': 8} if hasCuda else {}

# https://stackoverflow.com/questions/52439364/how-to-convert-rgb-images-to-grayscale-in-pytorch-dataloader
trainTransform  = transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()])

# https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder
train_percentage = .8
dataset = datasets.ImageFolder('./kkanji_data/', transform=trainTransform)

train_num = int(len(dataset) * train_percentage)
test_num = len(dataset) - train_num

train_set, test_set = torch.utils.data.random_split(dataset, [train_num, test_num])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, **kwargs)

# Model
model = VAE_CNN(1, h, z).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print('Running Epochs...')
for epoch in range(1, epochs + 1):
    train(epoch, log_interval)
    test(epoch, batch_size)
    if epoch % 5 == 0:
        with torch.no_grad():
            sample = torch.randn(64, z).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 64, 64),
                       'results_kanji/sample_kanji_cnn_big_dubs_' + str(epoch) + '.png')

torch.save(model.state_dict(), 'model.pt')

Running Epochs...
Train Epoch: 1 [0/112339 (0%)]	Loss: 2805.402100
Train Epoch: 1 [51200/112339 (46%)]	Loss: 1341.791260
Train Epoch: 1 [102400/112339 (91%)]	Loss: 1080.183838
====> Epoch: 1 Average loss: 1408.7898
====> Test set loss: 1064.4407
Train Epoch: 2 [0/112339 (0%)]	Loss: 1073.892456
Train Epoch: 2 [51200/112339 (46%)]	Loss: 972.312256
Train Epoch: 2 [102400/112339 (91%)]	Loss: 940.228149
====> Epoch: 2 Average loss: 979.7656
====> Test set loss: 923.8223
Train Epoch: 3 [0/112339 (0%)]	Loss: 870.819824
Train Epoch: 3 [51200/112339 (46%)]	Loss: 888.785889
Train Epoch: 3 [102400/112339 (91%)]	Loss: 819.815552
====> Epoch: 3 Average loss: 896.0293
====> Test set loss: 878.4152
Train Epoch: 4 [0/112339 (0%)]	Loss: 869.140930
Train Epoch: 4 [51200/112339 (46%)]	Loss: 840.709106
Train Epoch: 4 [102400/112339 (91%)]	Loss: 845.127563
====> Epoch: 4 Average loss: 857.8597
====> Test set loss: 852.6787
Train Epoch: 5 [0/112339 (0%)]	Loss: 906.836548
Train Epoch: 5 [51200/112339 (46%)]	